## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")

# Using the head function to showcase table sample
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pochutla,MX,15.7432,-96.4661,88.83,40,72,12.48,broken clouds
1,1,Kihei,US,20.7850,-156.4656,87.82,51,20,13.80,few clouds
2,2,Kodiak,US,57.7900,-152.4072,47.71,76,100,17.27,overcast clouds
3,3,Mount Gambier,AU,-37.8333,140.7667,40.91,100,13,0.00,few clouds
4,4,Kapaa,US,22.0752,-159.3190,80.58,78,20,9.22,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria

min_temp = float(input("What is the minimum temperature you would like for your trip?"))

max_temp = float(input("What is the maximum temperature you would like for your trip?"))


What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

# Using the head function to sample table

preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pochutla,MX,15.7432,-96.4661,88.83,40,72,12.48,broken clouds
1,1,Kihei,US,20.7850,-156.4656,87.82,51,20,13.80,few clouds
4,4,Kapaa,US,22.0752,-159.3190,80.58,78,20,9.22,light rain
5,5,Vaini,TO,-21.2000,-175.2000,80.76,78,20,6.91,few clouds
11,11,Alyangula,AU,-13.8483,136.4192,80.15,71,0,13.94,clear sky
15,15,Butaritari,KI,3.0707,172.7902,81.14,72,20,17.40,few clouds
18,18,Jamestown,US,42.0970,-79.2353,78.39,80,75,12.66,broken clouds
26,26,Riberalta,BO,-10.9833,-66.1000,80.80,41,26,0.00,scattered clouds
29,29,Russell,US,32.3502,-85.1999,83.80,57,0,4.61,clear sky
33,33,Cabo San Lucas,MX,22.8909,-109.9124,85.60,44,0,11.50,clear sky


In [6]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()


City_ID                193
City                   193
Country                193
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

clean_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pochutla,MX,15.7432,-96.4661,88.83,40,72,12.48,broken clouds
1,1,Kihei,US,20.7850,-156.4656,87.82,51,20,13.80,few clouds
4,4,Kapaa,US,22.0752,-159.3190,80.58,78,20,9.22,light rain
5,5,Vaini,TO,-21.2000,-175.2000,80.76,78,20,6.91,few clouds
11,11,Alyangula,AU,-13.8483,136.4192,80.15,71,0,13.94,clear sky


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pochutla,MX,88.83,broken clouds,15.7432,-96.4661,
1,Kihei,US,87.82,few clouds,20.7850,-156.4656,
4,Kapaa,US,80.58,light rain,22.0752,-159.3190,
5,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
11,Alyangula,AU,80.15,clear sky,-13.8483,136.4192,
15,Butaritari,KI,81.14,few clouds,3.0707,172.7902,
18,Jamestown,US,78.39,broken clouds,42.0970,-79.2353,
26,Riberalta,BO,80.80,scattered clouds,-10.9833,-66.1000,
29,Russell,US,83.80,clear sky,32.3502,-85.1999,
33,Cabo San Lucas,MX,85.60,clear sky,22.8909,-109.9124,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
    
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except:
        
        print("Hotel not found")
        

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found


In [31]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna()


In [32]:
# 8a. Create the output File (CSV)

output_data_file = "Vacation_Search/WeatherPY_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>

<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map.

locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))